In [61]:
from SPARQLWrapper import SPARQLWrapper2
import pandas as pds

## define function for converting sparql results to a dataframe

In [62]:
def sparql_to_df(results):
    data = {}
    for variable in results.variables:
        data[variable] = [x.value for x in results.getValues(variable)]
    return pds.DataFrame(data)

## define function for retrieving sparql query

In [63]:
def get_sparql(filename):
    with open(filename, 'r') as f:
        query = f.read()
        return query

## set connection to local triple store

In [64]:
sparql = SPARQLWrapper2("http://192.168.1.243:7200/repositories/EDR_NO_ES") # local server
# sparql = SPARQLWrapper2("http://10.16.128.24:7200/repositories/EDR_NO_ES") # RI server

## define some standard prefixes to use in queries

In [65]:
prefixes = get_sparql('queries/prefixes.sparql')
# print(prefixes)

## load patient info: this is needed b/c patients have been filtered for ages > 0, and we want to join our results on these patients

In [66]:
patient_df = pds.read_csv('triplestore-patient-info.tsv', sep='\t')

# Query/download data from triplestore

## find teeth (and their surfaces) that underwent a procedure

In [16]:
q = prefixes + get_sparql('queries/tooth-procedure-info.sparql')
# print(q)

In [17]:
sparql.setQuery(q)
res = sparql.query()
tooth_proc_df = sparql_to_df(res)

In [18]:
tooth_proc_df.drop_duplicates(inplace=True) # make sure there are not duplicates
len(tooth_proc_df)

1645416

## filter out invalid dates and convert datatypes (note: ignore warnings)

In [19]:
tooth_proc_df = tooth_proc_df[tooth_proc_df.event_date.str.startswith('19') | tooth_proc_df.event_date.str.startswith('20')]

In [20]:
tooth_proc_df.event_date = pds.to_datetime(tooth_proc_df.event_date)

In [21]:
tooth_proc_df['patient_id'] = tooth_proc_df['patient_id'].astype(str)
tooth_proc_df['tooth_id'] = tooth_proc_df['tooth_id'].astype(str)
tooth_proc_df['tooth_num'] = tooth_proc_df['tooth_num'].astype(str)
tooth_proc_df['cdt_code'] = tooth_proc_df['cdt_code'].astype(str)
tooth_proc_df['cdt_label'] = tooth_proc_df['cdt_label'].astype(str)

## join with patient info to filter out invalid patients

In [22]:
tooth_proc_df = pds.merge(tooth_proc_df, patient_df[['patient_id']], how='inner', on='patient_id')
len(tooth_proc_df)

1626907

## save local copy of tooth/procedure info

In [23]:
tooth_proc_df.to_csv('triplestore-tooth-procedure-info.tsv', sep='\t', index=False)

## find surfaces restored by procedures

In [67]:
q = prefixes + get_sparql('queries/procedure-surface-info.sparql')
# print(q)

In [68]:
sparql.setQuery(q)
res = sparql.query()
proc_surface_df = sparql_to_df(res)

In [69]:
proc_surface_df.drop_duplicates(inplace=True) # make sure there are no dups
len(proc_surface_df)

1252115

## join with patient info to filter out invalid patients

In [70]:
proc_surface_df = pds.merge(proc_surface_df, patient_df[['patient_id']], how='inner', on='patient_id')
len(proc_surface_df)

1235505

In [71]:
proc_surface_df.head()

,patient_id,tooth_id,tooth_num,proc_id,m,o,d,b,f,l,i
0,A_1_1_17,A_1_1_17_7,7,A_1_1_17_7_D2331_2011-04-23,1,0,0,0,0,0,1
1,A_1_1_17,A_1_1_17_9,9,A_1_1_17_9_D2335_2006-07-07,1,0,0,0,0,0,1
2,A_1_1_17,A_1_1_17_10,10,A_1_1_17_10_D2330_2008-10-12,1,0,0,0,0,0,0
3,A_1_1_17,A_1_1_17_10,10,A_1_1_17_10_D2335_2014-03-16,1,0,0,0,1,1,1
4,A_1_1_17,A_1_1_17_12,12,A_1_1_17_12_D2392_2012-01-23,0,1,1,0,0,0,0


## add colum with total number of surfaces restored

In [72]:
proc_surface_df['m'] = proc_surface_df['m'].astype(int) 
proc_surface_df['o'] = proc_surface_df['o'].astype(int)
proc_surface_df['d'] = proc_surface_df['d'].astype(int)
proc_surface_df['b'] = proc_surface_df['b'].astype(int)
proc_surface_df['f'] = proc_surface_df['f'].astype(int)
proc_surface_df['l'] = proc_surface_df['l'].astype(int) 
proc_surface_df['i'] = proc_surface_df['i'].astype(int)

In [73]:
proc_surface_df['num_surfaces'] = \
    proc_surface_df['m'] + proc_surface_df['o'] + proc_surface_df['d'] + \
    proc_surface_df['b'] + proc_surface_df['f'] + proc_surface_df['l'] + proc_surface_df['i']

## save local copy of procedure/surface info

In [76]:
proc_surface_df.to_csv('triplestore-procedure-surface-info.tsv', sep='\t', index=False)

## find tooth type metadata for each tooth

In [29]:
q = prefixes + get_sparql('queries/tooth-type-info.sparql')
# print(q)

In [30]:
sparql.setQuery(q)
res = sparql.query()
tooth_type_df = sparql_to_df(res)

In [31]:
tooth_type_df = pds.read_csv('triplestore-tooth-type-info.tsv', sep='\t')

In [32]:
tooth_type_df.drop_duplicates(inplace=True) # make sure there are no dups
len(tooth_type_df)

1210430

## add maxillary maxilla (1-16) mandible (17-32) metadata

In [33]:
tooth_type_df.tooth_num = tooth_type_df.tooth_num.astype(int) # convert tooth_num to int
tooth_type_df['maxillary'] = tooth_type_df.tooth_num.map(lambda x: 1 if x < 17 else 0)
tooth_type_df['mandibular'] = tooth_type_df.tooth_num.map(lambda x: 1 if x > 16 else 0)

## add left/right metadata

In [34]:
tooth_type_df['left'] = tooth_type_df.tooth_num.map(lambda x: 1 if x > 8 and x < 25 else 0)
tooth_type_df['right'] = tooth_type_df.tooth_num.map(lambda x: 1 if x < 9 or x > 24 else 0)

## join with patient info to filter out invalid patients

In [35]:
tooth_type_df = pds.merge(tooth_type_df, patient_df[['patient_id']], how='inner', on='patient_id')
len(tooth_type_df)

1194463

## save local copy of tooth type metadata info

In [36]:
tooth_type_df.to_csv('triplestore-tooth-type-info.tsv', sep='\t', index=False)

## find materials used crowns (inluding onlays), fillings (including inlays), and veneers

In [77]:
q = prefixes + get_sparql('queries/proc-material-info.sparql')
# print(q)

In [78]:
sparql.setQuery(q)
res = sparql.query()
proc_material_df = sparql_to_df(res)

In [79]:
proc_material_df.drop_duplicates(inplace=True) # make sure there are no dups
len(proc_material_df)

1686822

## join with patient info to filter out invalid patients

In [80]:
proc_material_df = pds.merge(proc_material_df, patient_df[['patient_id']], how='inner', on='patient_id')
len(proc_material_df)

1668204

## save local copy of procedure/material info

In [81]:
proc_material_df.to_csv('triplestore-proc-material-info.tsv', sep='\t', index=False)